In [2]:
"""Postpartum Depression Classification with BETR/PsghGPT2/T5"""
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# === Parameters ===
MODEL_NAME = 'roberta-large'
NUM_LABELS = 3
MAX_LENGTH = 512
BATCH_SIZE = 8
EPOCHS = 10
TFIDF_DIM = 1000
LR = 1e-5

# === Load Data ===
df = pd.read_csv("ppd_data.csv")

# === Tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_text(text):
    return tokenizer.encode_plus(
        text,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        return_attention_mask=True
    )

# === TF-IDF Features ===
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=TFIDF_DIM)
tfidf_features = tfidf_vectorizer.fit_transform(df['text']).toarray()

# === Tokenization ===
input_ids, attention_masks = [], []
for text in df['text']:
    encoded = tokenize_text(text)
    input_ids.append(encoded['input_ids'].squeeze(0))
    attention_masks.append(encoded['attention_mask'].squeeze(0))

# === Dataset ===
class FusionDataset(Dataset):
    def __init__(self, input_ids, attention_masks, tfidf_features, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.tfidf_features = tfidf_features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'tfidf': torch.tensor(self.tfidf_features[idx], dtype=torch.float),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# === Train/Val Split ===
train_idx, temp_idx = train_test_split(np.arange(len(df)), test_size=0.2, stratify=df['label'])
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, stratify=df.loc[temp_idx, 'label'])


train_dataset = FusionDataset(
    [input_ids[i] for i in train_idx],
    [attention_masks[i] for i in train_idx],
    tfidf_features[train_idx],
    df['label'].values[train_idx]
)

val_dataset = FusionDataset(
    [input_ids[i] for i in val_idx],
    [attention_masks[i] for i in val_idx],
    tfidf_features[val_idx],
    df['label'].values[val_idx]
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# === Model ===
class BertWithTFIDF(nn.Module):
    def __init__(self, model_name, tfidf_dim, num_labels):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Linear(hidden_size + tfidf_dim, num_labels)

    def forward(self, input_ids, attention_mask, tfidf_features):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        fused = torch.cat((cls_output, tfidf_features), dim=1)
        fused = self.dropout(fused)
        return self.classifier(fused)

# === Train ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertWithTFIDF(MODEL_NAME, TFIDF_DIM, NUM_LABELS).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        tfidf = batch['tfidf'].to(device)
        labels = batch['labels'].to(device)

        logits = model(input_ids, attention_mask, tfidf)
        loss = loss_fn(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"[Epoch {epoch+1}] Train Loss: {total_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    val_preds, val_labels, val_loss = [], [], 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            tfidf = batch['tfidf'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask, tfidf)
            loss = loss_fn(logits, labels)
            val_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(val_labels, val_preds)
    f1 = f1_score(val_labels, val_preds, average='weighted')
    print(f"[Epoch {epoch+1}] Val Loss: {val_loss / len(val_loader):.4f} | Acc: {acc:.4f} | Weighted F1: {f1:.4f}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Epoch 1] Train Loss: 0.9764
[Epoch 1] Val Loss: 0.9546 | Acc: 0.6562 | Weighted F1: 0.6015
[Epoch 2] Train Loss: 0.8538
[Epoch 2] Val Loss: 0.6945 | Acc: 0.7292 | Weighted F1: 0.7182
[Epoch 3] Train Loss: 0.6091
[Epoch 3] Val Loss: 0.7314 | Acc: 0.6667 | Weighted F1: 0.6494
[Epoch 4] Train Loss: 0.4571
[Epoch 4] Val Loss: 0.6628 | Acc: 0.7292 | Weighted F1: 0.7009
[Epoch 5] Train Loss: 0.2676
[Epoch 5] Val Loss: 0.7593 | Acc: 0.6667 | Weighted F1: 0.6802
[Epoch 6] Train Loss: 0.1732
[Epoch 6] Val Loss: 0.9875 | Acc: 0.6979 | Weighted F1: 0.6993
[Epoch 7] Train Loss: 0.0882
[Epoch 7] Val Loss: 0.9444 | Acc: 0.6979 | Weighted F1: 0.6985
[Epoch 8] Train Loss: 0.0951
[Epoch 8] Val Loss: 0.9651 | Acc: 0.7292 | Weighted F1: 0.7302
[Epoch 9] Train Loss: 0.0254
[Epoch 9] Val Loss: 1.2518 | Acc: 0.7396 | Weighted F1: 0.7347
[Epoch 10] Train Loss: 0.0101
[Epoch 10] Val Loss: 1.3794 | Acc: 0.6979 | Weighted F1: 0.6955


✅ 第一步：保存 TF-IDF 向量器
python
Copy
Edit
import joblib

# 保存 TF-IDF 向量器
joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.pkl")
⏬ 生成文件：tfidf_vectorizer.pkl
➡️ 作用：后续接收到新文本时，可以用它将文本转为 TF-IDF 特征向量。
✅ 第二步：保存 BERT+TFIDF 融合模型
你自定义了 BertWithTFIDF，这是非 HuggingFace 原生的模型类，所以不能用 model.save_pretrained 来保存，而应该使用 PyTorch 的保存方式：

python
Copy
Edit
# 保存模型结构的类（FastAPI 中要复现）
torch.save(model.state_dict(), "bert_tfidf_fusion_model.pt")
⏬ 生成文件：bert_tfidf_fusion_model.pt
➡️ 作用：保存你 fine-tuned 后模型的所有权重参数

✅ 第三步：保存 tokenizer
你仍然需要保存 HuggingFace 的 tokenizer，因为后续也要对输入文本进行编码：

python
Copy
Edit
tokenizer.save_pretrained("tokenizer/")
⏬ 生成文件夹：tokenizer/，其中包含：

vocab.json

merges.txt

tokenizer_config.json

special_tokens_map.json

✅ 最终你需要保留的文件结构如下：
pgsql
Copy
Edit
models/
├── tfidf_vectorizer.pkl             ← TF-IDF 向量器
├── bert_tfidf_fusion_model.pt       ← 融合模型权重
└── tokenizer/                       ← RoBERTa tokenizer
    ├── vocab.json
    ├── merges.txt
    ├── tokenizer_config.json
    └── special_tokens_map.json

🧱 第一步：创建你的 FastAPI 项目结构
在你的本地（或服务器）某个文件夹下，创建如下结构：

pgsql
Copy
Edit
ppd_api_project/
│
├── main.py                    ← FastAPI 主入口
├── model.py                   ← 定义 BertWithTFIDF 类
├── requirements.txt           ← 依赖库列表
└── models/                    ← 模型和向量器文件夹
    ├── tfidf_vectorizer.pkl
    ├── bert_tfidf_fusion_model.pt
    └── tokenizer/
        ├── vocab.json
        ├── merges.txt
        ├── tokenizer_config.json
        └── special_tokens_map.json


In [3]:
import joblib

# 保存 TF-IDF 向量器
joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']

In [4]:
# 保存模型结构的类（FastAPI 中要复现）
torch.save(model.state_dict(), "bert_tfidf_fusion_model.pt")
tokenizer.save_pretrained("tokenizer/")


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.json',
 'tokenizer/merges.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [5]:
# === Tokenization ===
input_ids, attention_masks = [], []
for text in df['text']:
    encoded = tokenize_text(text)
    input_ids.append(encoded['input_ids'].squeeze(0))
    attention_masks.append(encoded['attention_mask'].squeeze(0))

test_dataset = FusionDataset(
    [input_ids[i] for i in test_idx],
    [attention_masks[i] for i in test_idx],
    tfidf_features[test_idx],
    df['label'].values[test_idx]
)


test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
model.eval()
test_preds, test_labels, test_loss = [], [], 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        tfidf = batch['tfidf'].to(device)
        labels = batch['labels'].to(device)

        logits = model(input_ids, attention_mask, tfidf)
        loss = loss_fn(logits, labels)
        test_loss += loss.item()

        preds = torch.argmax(logits, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_acc = accuracy_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds, average='weighted')
print(f"[Test] Loss: {test_loss / len(test_loader):.4f} | Acc: {test_acc:.4f} | Weighted F1: {test_f1:.4f}")


[Test] Loss: 1.0448 | Acc: 0.7917 | Weighted F1: 0.7917


In [6]:
def compute_metrics(preds, true_labels):
    return {
        "accuracy": accuracy_score(true_labels, preds),
        "f1_weighted": f1_score(true_labels, preds, average="weighted"),
        "f1_macro": f1_score(true_labels, preds, average="macro")
    }
metrics = compute_metrics(test_preds, test_labels)

print(f"[Test Metrics]")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"F1 (Weighted): {metrics['f1_weighted']:.4f}")
print(f"F1 (Macro): {metrics['f1_macro']:.4f}")


[Test Metrics]
Accuracy: 0.7917
F1 (Weighted): 0.7917
F1 (Macro): 0.7622


In [7]:
def predict_rating(text):
    model.eval()
    with torch.no_grad():
        # Tokenizer
        encoded = tokenizer.encode_plus(
            text,
            max_length=MAX_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True
        )
        input_ids = encoded['input_ids'].to(device)
        attention_mask = encoded['attention_mask'].to(device)

        # TF-IDF
        tfidf_feat = tfidf_vectorizer.transform([text]).toarray()
        tfidf_tensor = torch.tensor(tfidf_feat, dtype=torch.float).to(device)

        # Predict
        logits = model(input_ids, attention_mask, tfidf_tensor)
        pred = torch.argmax(logits, dim=1).item()
        return f"Predicted Rating: {pred}"
